In [1]:
import pandas as pd
import numpy as np
from xbbg import blp

In [2]:
DO_HISTORY = False

***

In [3]:
TICKS = ['CT10', 'CB3', 'CTII10']
TYPE = 'Govt'
bbcodes = [f'{tick} {TYPE}' for tick in TICKS]
FLDS = ['NAME','SECURITY_TYP','MATURITY','MTY_YEARS','CPN','INT_ACC','AMT_OUTSTANDING','BID_YIELD','PX_BID','QT_SPEC','ISSUE_DT','IS_US_ON_THE_RUN','PRINCIPAL_FACTOR']
info = blp.bdp(bbcodes,FLDS)

In [4]:
def decimal_to_32nds(price: float) -> str:
    """
    Convert a decimal price to a US Treasury note quote in 32nds.

    Parameters
    ----------
    price : float
        Price in decimal (e.g., 100.234375)

    Returns
    -------
    str
        Quote string in the format "AAA-BB" or "AAA-BB+" if there's a half 32nd.

    Examples
    --------
    >>> decimal_to_32nds(100.15625)
    '100-05+'
    >>> decimal_to_32nds(99.96875)
    '99-31+'
    >>> decimal_to_32nds(101.0)
    '101-00'
    """
    # Split into whole number and fractional part
    whole = int(price)
    frac = price - whole

    # Convert fractional part to 64ths and round to nearest
    total_64ths = round(frac * 64)

    # Determine number of whole 32nds and if there is a half 32nd
    n32 = total_64ths // 2
    half_flag = total_64ths % 2

    # Handle rollover if rounding pushes to the next whole point
    if n32 == 32:
        whole += 1
        n32 = 0
        half_flag = 0

    # Build the quote string
    quote = f"{whole}-{n32:02d}"
    if half_flag:
        quote += "+"
    return quote

In [5]:
ticks_32nds = ['CT10 Govt','CTII10 Govt']
for tick in ticks_32nds:
    info.loc[tick,'px_bid'] = decimal_to_32nds(info.loc[tick,'px_bid'])
info['cpn'] /= 100

In [6]:
info.rename(columns={'amt_outstanding':'amount issued','principal_factor':'inflation adjustment','issue_dt':'issue date','bid_yield':'ytm (bid)','maturity':'maturity date','px_bid':'price quoted (bid)','mty_years':'ttm (years)','int_acc':'accrued interest','cpn':'coupon rate','qt_spec':'quote specification'}, inplace=True)
info

,name,security_typ,maturity date,ttm (years),coupon rate,accrued interest,amount issued,ytm (bid),price quoted (bid),quote specification,issue date,is_us_on_the_run,inflation adjustment
CB3 Govt,TREASURY BILL,US GOVERNMENT,2025-10-02,0.238193,0.0,0.0,207649000000.0,4.362,4.2575,4,2024-10-03,Y,1.0
CT10 Govt,US TREASURY N/B,US GOVERNMENT,2035-05-15,9.853525,0.0425,0.612092,96526000000.0,4.35,99-06+,512ths,2025-05-15,Y,1.0
CTII10 Govt,TSY INFL IX N/B,US GOVERNMENT,2035-01-15,9.524983,0.02125,1.015539,57625000000.0,1.999,101-03,512ths,2025-01-31,N,1.01691


In [7]:
FLDS_EXPORT = ['name','issue date','maturity date', 'amount issued', 'ttm (years)', 'inflation adjustment', 'coupon rate','accrued interest','price quoted (bid)','ytm (bid)']
info_export = info[FLDS_EXPORT]
info_export

,name,issue date,maturity date,amount issued,ttm (years),inflation adjustment,coupon rate,accrued interest,price quoted (bid),ytm (bid)
CB3 Govt,TREASURY BILL,2024-10-03,2025-10-02,207649000000.0,0.238193,1.0,0.0,0.0,4.2575,4.362
CT10 Govt,US TREASURY N/B,2025-05-15,2035-05-15,96526000000.0,9.853525,1.0,0.0425,0.612092,99-06+,4.35
CTII10 Govt,TSY INFL IX N/B,2025-01-31,2035-01-15,57625000000.0,9.524983,1.01691,0.02125,1.015539,101-03,1.999


***

In [8]:
if DO_HISTORY:
    DATESTART = '2000-01-01'
    DATEEND = '2025-05-31'

    FLDS = ['LAST_PRICE']
        
    ts = blp.bdh(bbcodes, FLDS, start_date=DATESTART,end_date=DATEEND)


    ts.index = pd.to_datetime(ts.index)
    ts.index.name = 'date'
    ts = ts.xs('LAST_PRICE', axis=1, level=1)
    ts.dropna(inplace=True)

    ts_export = ts.copy()

***

# Export Data

In [9]:
OUTFILE = f'../data/exam_3_data.xlsx'

with pd.ExcelWriter(OUTFILE) as writer:  
    info_export.T.to_excel(writer, sheet_name= f'treasury specs', index=True)
    if DO_HISTORY:
        ts_export.to_excel(writer, sheet_name= f'treasury price history', index=True)


06JUL2025_10:55:13.714 53690:6206500864 WARN apism_apisession.cpp:1332 Apisession.aliveCheck Alive Request Retries exhausted (2) on [localhost:8194 -> 127.0.0.1:65518] 

06JUL2025_10:55:13.715 53690:13288632320 WARN blpapi_platformcontroller.cpp:489 blpapi.session.{1}.platformcontroller Connectivity lost, no connected endpoints. 

06JUL2025_10:55:21.013 53690:6207074304 WARN blpapi_apicmadapter.cpp:358 blpapi.session.{1}.transporttcp.apicmadapter Connection attempt timed out localhost:8194 <127.0.0.1:8194> 

06JUL2025_10:55:21.013 53690:6207074304 WARN blpapi_platformtransporttcp.cpp:138 blpapi.session.{1}.transporttcp<0><localhost:8194> 127.0.0.1, session pool state=Failed 

06JUL2025_10:59:21.707 53690:6207074304 WARN apicm_apiconnector.cpp:571 ApiConnector::negotiatorCallback localhost: Session negotiation failed for 127.0.0.1:8194 result = 1 

06JUL2025_10:59:21.707 53690:6207074304 WARN blpapi_apicmadapter.cpp:350 blpapi.session.{1}.transporttcp.apicmadapter Failed BBCOMM session